In [ ]:
############
#
# _____                              _____    ___      _____                     __          __    _     _                _____          _             
# / ____| __       __  |   __ |      |  __ \  /   \    |  __ \         __         \ \        / /   ( ) | ( )|\ |  __      / ____| __     ( )    |       
#| (___  /  \|__  /  \_|_ /  \|__    | |__) |/  ^  \   | |  | |     ◠ /  \ |__     \ \  /\  / /|__  | _|_ | | \| /  \    | (___  /  \|__  | |___|_ __   
# \___ \|    |  \| () || |    |  \   |  ___//  /_\  \  | |  | |    | | () ||  \     \ \/  \/ / |  \ |  |  | |  || () |    \___ \|    |  \ | |  \| /__   
# ____) |\__/|    \__/ |  \__/|  |   | |   /  _____  \ | |__| |   ¯T¯ \__/ |         \  /\  /  |    |  |  | |  | \__/     ____) |\__/|    | |__/|  __/  
#|_____/             \ |_            |_|  /__/     \__\|_____/     |                  \/  \/       (_) |_(_)        |    |_____/         (_)|   |_      
#                                         |__|     |__|            |                                             \__/                       |           
#                                                                                  
###########

In [ ]:
import pandas as pd

dfi = pd.read_csv("/home/gridsan/akolodziej/01_insert_domains/outputs/parent_insert_duplications.tsv", sep = '\t')

In [ ]:
########
# Script to align insert/parent domains using TM align with and without -cp
# Use to see if inserts are cps of parent domains
# USE CONDA ENVIRONMENT WITH FOLDCOMP INSTALLED: prog_mod WRITE IN THE RULE USING THIS SCRIPT!!!!!
##########

import sys
import csv
import os
import pandas as pd
import foldcomp
from pdbtools.pdb_selres_str import select_residues_from_string
sys.path.append('/home/gridsan/akolodziej/TM_tools/')
from TM_utils import TM_align

In [ ]:
def convert_boundaries(input_str):
    ''' convert boundary str from foldcomp to one that works for pdbtools'''
    # Replace underscores with commas and hyphens with colons
    result = input_str.replace('_', ',').replace('-', ':')
    return result

In [ ]:
data = '/home/gridsan/akolodziej/01_insert_domains/outputs/parent_insert_duplications.tsv'

In [ ]:
out_file = 'parent_insert_CP_scores.tsv'

df = pd.read_csv(data, sep ='\t')

In [ ]:


af_name_bounds = {}
# loop through df
z = 0
for index, row in df.iterrows():
    # Get parent_ted field
    parent_ted = row['parent_ted']
    # Remove the _TED suffix and anything after
    af_name = parent_ted.split('_TED')[0]
    # Get indices of the domains
    parent_bounds = row['parent_boundaries']
    insert_bounds = row['insert_boundaries']
    # Convert the boundaries to a format that pdbtools can use
    parent_bounds_cor = convert_boundaries(parent_bounds)
    insert_bounds_cor = convert_boundaries(insert_bounds)
    print(f'Parsing {af_name}')
    print(f'Corrected bounds for pdbtools parsing: {parent_bounds_cor}, {insert_bounds_cor}')

    bounds = [parent_bounds_cor,insert_bounds_cor]
    af_name_bounds[af_name] = bounds
    # Check to make sure the af_name is within the foldcomp database
    # TO DO
    # Is af_name within the foldcomp index? if not, continue to next row
    # TO DO
    # Use foldcomp to get the structure 
    z+=1
    if z == 10:
        break

names = list(af_name_bounds.keys())

In [ ]:
af_name_bounds['AF-A0A009ENQ2-F1-model_v4'][0]

In [ ]:
        # Extract the two domains from the pdb
        # Use pdbtools to parse into domains
        parent_domain = select_residues_from_string(pdb, parent_bounds_cor)
        insert_domain = select_residues_from_string(pdb, insert_bounds_cor)
        # Write the two domains to pdb files
        parent_pdb = f'/home/gridsan/akolodziej/01_insert_domains/outputs/pdbs/{af_name}_parent.pdb'
        insert_pdb = f'/home/gridsan/akolodziej/01_insert_domains/outputs/pdbs/{af_name}_insert.pdb'
        if not os.path.exists(parent_pdb):
                with open(parent_pdb, "w") as f:
                    f.write(parent_domain)
        if not os.path.exists(insert_pdb):
                with open(insert_pdb, "w") as f:
                    f.write(insert_domain)

In [ ]:
df_CATH = dfi[(dfi['insert_cath'] == "3.40.640.10")]
              #& (dfi['insert_cath'] == "2.30.42.40")]


In [ ]:
df_CATH

In [1]:
import os

In [2]:
def set_trained_model_fp(model):
    '''Function to update progres model'''
    if model_name == 'CIRPIN':
        os.environ["TRAINED_MODEL_FP"] = "//home/gridsan/akolodziej/progres_link/trained_models/CIRPIN/CIRPIN_model/model_5k_cp_epoch301.pt"
    if model_name == 'Progres':
        os.environ["TRAINED_MODEL_FP"] = "/home/gridsan/akolodziej/progres_link/trained_models/v_0_2_0/trained_model.pt"
    else:
        pass
    print(f"Updated trained_model_fp to: {os.environ['TRAINED_MODEL_FP']}")


## Define model to use:
model_name = 'CIRPIN'
# Call set trained_model_fp
set_trained_model_fp(model_name)

Updated trained_model_fp to: //home/gridsan/akolodziej/progres_link/trained_models/CIRPIN/CIRPIN_model/model_5k_cp_epoch301.pt


In [3]:
import progres as pg

/home/gridsan/akolodziej/.conda/envs/prog_mod/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
pdb1 = pg.embed_structure('/home/gridsan/akolodziej/A0A4S8SQP1.pdb')

Using model model_5k_cp_epoch301.pt


In [12]:
pdb2 = pg.embed_structure('/home/gridsan/akolodziej/A0A1F9ASQ9_insert.pdb')

Using model model_5k_cp_epoch301.pt


In [13]:
score = pg.embedding_similarity(pdb1,pdb2)

In [14]:
score

tensor(0.9299)

In [ ]:
def progres_search_print(querystructure=None, querylist=None, queryembeddings=None, targetdb=None,
                         fileformat="guess", minsimilarity=default_minsimilarity,
                         maxhits=default_maxhits, chainsaw=False, device="cpu", batch_size=None, trained_model=trained_model_fp):

In [9]:
search_score = pg.progres_search(queryembeddings=pdb1,
                                 targetdb='/home/gridsan/akolodziej/progres_link/databases/ark_scope40_Progres_embed_1_7_25.pt')

TRAINED MODEL //home/gridsan/akolodziej/progres_link/trained_models/CIRPIN/CIRPIN_model/model_5k_cp_epoch301.pt
Using model model_5k_cp_epoch301.pt


In [10]:
search_score

[{'query_num': 1,
  'query': '?',
  'domain_num': 1,
  'domain_size': '?',
  'res_range': '?',
  'database': '/home/gridsan/akolodziej/progres_link/databases/ark_scope40_Progres_embed_1_7_25.pt',
  'minsimilarity': 0.8,
  'maxhits': 100,
  'domains': [],
  'hits_nres': [],
  'similarities': [],
  'notes': []}]